In [ ]:
from tqdm.auto import tqdm

### pdb file preprocessing related code

In [ ]:
import numpy as np

with open("/Data/deeksha/disha/ProtTrans/data/list_file.txt") as f:
    data = f.read().split(",")
len(data)
data = np.array(data)
np.unique(data).shape

In [ ]:
import os
'''
Extract the files which were left to download and store them to new file.           
'''
f_names = os.listdir("/Data/deeksha/disha/pdbid_files")
new_fn_name = []
for f_name in f_names:
    _fname = f_name.split('.')[0]
    new_fn_name.append(_fname)
len(new_fn_name), new_fn_name[0]

data, new_fn_name = set(data), set(new_fn_name)

unique_f_name = list(data.difference(new_fn_name))
print(len(unique_f_name))
unique_f_name[0]

with open("/Data/deeksha/disha/ProtTrans/scripts/list_file2.txt", 'w') as f2:
    f2.write(",".join(unique_f_name))

In [ ]:
import pickle as pkl

with open("/Data/deeksha/disha/ProtTrans/data/dic_seq.pkl", 'rb') as f:
    data = pkl.load(f)
len(data.keys())


In [ ]:
data['1wd3']['MET'][-1]
# data.keys()

--------------------------------------------------------------------------------------------------------
### DF-Final code 
--------------------------------------------------------------------------------------------------------


In [ ]:
import numpy as np

# load npz file
casp12 = np.load('/Data/deeksha/disha/ProtTrans/data/dataset_npz_files/CASP12_HHblits.npz', allow_pickle=True)
cb513 = np.load("/Data/deeksha/disha/ProtTrans/data/dataset_npz_files/CB513_HHblits.npz", allow_pickle=True)
train = np.load("/Data/deeksha/disha/ProtTrans/data/dataset_npz_files/Train_HHblits.npz", allow_pickle=True)
ts115 = np.load("/Data/deeksha/disha/ProtTrans/data/dataset_npz_files/TS115_HHblits.npz", allow_pickle=True)

# extract data
casp12_data = casp12['data']
cb513_data = cb513['data']
train_data = train['data']
ts115_data = ts115['data']

casp12_pdb = casp12['pdbids']
cb513_pdb = cb513['pdbids']
train_pdb = train['pdbids']
ts115_pdb = ts115['pdbids']

# print shapes of all data

In [ ]:
# print shapes of all data
print(casp12_data.shape)
# print(casp12_pdb.shape)
print(cb513_data.shape)
# print(cb513_pdb.shape)
print(train_data.shape)
# print(train_pdb.shape)
print(ts115_data.shape)
# print(ts115_pdb.shape)

In [3]:
# extract [0:20] Amino Acids and [57:65] Q8 codes
primary_casp12 = casp12_data[:,:, :20]
q8_casp12 = casp12_data[:,:, 57:65]

primary_cb513 = cb513_data[:,:, :20]
q8_cb513 = cb513_data[:,:, 57:65]

primary_train = train_data[:,:, :20]
q8_train = train_data[:,:, 57:65]

primary_ts115 = ts115_data[:,:, :20]
q8_ts115 = ts115_data[:,:, 57:65]

In [ ]:
primary_train.shape, q8_train.shape

#### Encoding primary and secondary sequence from one-hot to alphabets

In [ ]:
# B', 'C', 'E', 'G', 'H', 'I', 'S', 'T'
int2second = {0: 'G', 1: 'H', 2: 'I', 3: 'B', 4: 'E', 5: 'S', 6: 'T', 7: 'C'}
encoded_second_seqs, max_sample_len = [], []

for ex in tqdm(q8_train):
    seq = ""
    for aa in ex:
        if sum(aa) == 0:
            # seq += "X"
            continue
        else:
            seq += int2second[np.argmax(aa)]
    max_sample_len.append(len(seq))
    encoded_second_seqs.append(seq)

# encoded_second_seqs = np.array(encoded_second_seqs)
# encoded_second_seqs[10:30]

In [ ]:
int2primary = {0: 'A', 1: 'C', 2: 'D', 3: 'E', 4: 'F', 5: 'G', 6: 'H', 7: 'I', 8: 'K', 9: 'L', 10: 'M', 11: 'N', 12: 'P', 13: 'Q', 14: 'R', 15: 'S', 16: 'T', 17: 'V', 18: 'W', 19: 'Y', 20: 'X'}

# {0: 'H', 1: 'B', 2: 'E', 3: 'G', 4: 'I', 5: 'T', 6: 'S', 7: 'C', 8:'X'}
encoded_primary_seqs = []

for i, ex in enumerate(tqdm(primary_train)):
    ex = ex[:max_sample_len[i]]
    seq = ""
    for i, aa in enumerate(ex):
        if sum(aa) == 0:
            seq += 'X'
            # continue
        else:
            seq += int2primary[np.argmax(aa)]
    encoded_primary_seqs.append(seq)

# encoded_primary_seqs = np.array(encoded_primary_seqs)
encoded_primary_seqs[:10]

In [ ]:
encoded_primary_seqs[0], encoded_second_seqs[0]

#### Converting to dataframe

In [ ]:
import pandas as pd

df = pd.DataFrame({"input": encoded_primary_seqs, "dssp8": encoded_second_seqs})
df['input'] = df.apply(lambda x: " ".join(list(x['input'])), axis=1)
df['dssp8'] = df.apply(lambda x: " ".join(list(x['dssp8'])), axis=1)
df['pdbid'] = train_pdb
df['pdbid'] = df.apply(lambda x: x['pdbid'].split('-')[0], axis=1)
df.head()

In [ ]:
train_path = "/Data/deeksha/disha/ProtTrans/data/csv_files/Train_HHblits.csv"
cb513_path = "/Data/deeksha/disha/ProtTrans/data/csv_files/CB513_HHblits.csv"
ts115_path = "/Data/deeksha/disha/ProtTrans/data/csv_files/TS115_HHblits.csv"
casp12_path = "/Data/deeksha/disha/ProtTrans/data/csv_files/CASP12_HHblits.csv"

df_train_alpha = pd.read_csv(train_path)
# df_train_alpha = df_train_alpha[['input', ' dssp8']]
df_train_alpha.head()

In [ ]:
print(f'shape of df_train_alpha: {df_train_alpha.shape}')
print(f'shape of df: {df.shape}')

In [ ]:
df_train_alpha[' dssp8'].isin(df['dssp8']).value_counts()

In [ ]:
df_train_alpha.rename(columns={' dssp8': 'dssp8'}, inplace=True)
df = df.drop_duplicates(subset=['dssp8'])
df_train_alpha = df_train_alpha.drop_duplicates(subset=['dssp8'])
df.shape, df.dssp8.nunique(), df.input.nunique(), df_train_alpha.shape, df_train_alpha['dssp8'].nunique(), df_train_alpha.input.nunique()

In [ ]:

df3 = pd.merge(df_train_alpha, df, on=['dssp8'], how='inner')
df3.head()

In [ ]:
df3 = df3.drop_duplicates(subset=['dssp8'])
df3.shape

In [ ]:
(df3.input_x == df3.input_y).value_counts()

#### Saving final dataframe

In [101]:
train_path = "/Data/deeksha/disha/ProtTrans/data/final_excel_files/df_final.xlsx"
cb513_path = "/Data/deeksha/disha/ProtTrans/data/final_excel_files/cb513_final.xlsx"
ts115_path = "/Data/deeksha/disha/ProtTrans/data/final_excel_files/ts115_final.xlsx"
casp12_path = "/Data/deeksha/disha/ProtTrans/data/final_excel_files/casp12_final.xlsx"

df3.to_excel(train_path, index=False)

In [ ]:
df3.shape

In [4]:
import pandas as pd

df = pd.read_excel("/Data/deeksha/disha/ProtTrans/data/final_excel_files/df_final.xlsx")

df_train = df[:int(0.9*df.shape[0])]
df_test = df[int(0.9*df.shape[0]):]

# save train and test data
df_train.to_excel("/Data/deeksha/disha/ProtTrans/data/final_excel_files/df_train.xlsx", index=False)
df_test.to_excel("/Data/deeksha/disha/ProtTrans/data/final_excel_files/df_test.xlsx", index=False)